In [6]:
import pandas as pd

df = pd.read_csv("FinMark_Cleaned.csv")
df.head()
df.info()
df.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Customer_ID              25000 non-null  int64  
 1   Satisfaction_Score       25000 non-null  float64
 2   Feedback_Comments        25000 non-null  object 
 3   Likelihood_to_Recommend  25000 non-null  int64  
 4   Transaction_ID           24966 non-null  float64
 5   Transaction_Date         24966 non-null  object 
 6   Transaction_Amount       24966 non-null  float64
 7   Transaction_Type         24966 non-null  object 
 8   FirstTransactionDate     24966 non-null  object 
 9   AccountAgeDays           24966 non-null  float64
dtypes: float64(4), int64(2), object(4)
memory usage: 1.9+ MB


,Customer_ID,Satisfaction_Score,Likelihood_to_Recommend,Transaction_ID,Transaction_Amount,AccountAgeDays
count,25000.000000,25000.000000,25000.000000,24966.000000,24966.000000,24966.000000
mean,506.820840,5.690313,5.526560,2510.686734,3092.749619,1088.477369
std,291.870887,3.567779,2.872188,1450.481496,14735.669434,32.102942
min,1.000000,1.000000,1.000000,1.000000,10.000000,915.000000
25%,255.000000,3.000000,3.000000,1251.000000,1252.000000,1075.000000
50%,509.000000,6.000000,6.000000,2521.000000,2483.000000,1098.000000
75%,766.000000,8.000000,8.000000,3768.000000,3660.000000,1112.000000
max,1000.000000,60.000000,10.000000,5000.000000,480300.000000,1122.000000


In [11]:
# Missing values
print(df.isnull().sum())

# Duplicates
print(df.duplicated().sum())

print(df['Transaction_Type'].unique())


Customer_ID                 0
Satisfaction_Score          0
Feedback_Comments           0
Likelihood_to_Recommend     0
Transaction_ID             34
Transaction_Date           34
Transaction_Amount         34
Transaction_Type           34
FirstTransactionDate       34
AccountAgeDays             34
dtype: int64
0
['Loan Payment' 'Bill Payment' 'Investment' 'Purchase' nan]


In [14]:
df['Transaction_ID'] = df['Transaction_ID'].fillna(0)
df['Transaction_Amount'] = df['Transaction_Amount'].fillna(0)
df['Transaction_Type'] = df['Transaction_Type'].fillna('No Transaction')


In [15]:
df = df.dropna(subset=['Transaction_ID'])


In [16]:
df.isnull().sum()


Customer_ID                 0
Satisfaction_Score          0
Feedback_Comments           0
Likelihood_to_Recommend     0
Transaction_ID              0
Transaction_Date           34
Transaction_Amount          0
Transaction_Type            0
FirstTransactionDate       34
AccountAgeDays              0
dtype: int64

In [18]:
df['Transaction_Date'] = df['Transaction_Date'].fillna(pd.Timestamp('1970-01-01'))
df['FirstTransactionDate'] = df['FirstTransactionDate'].fillna(pd.Timestamp('1970-01-01'))


In [19]:
df['AccountAgeDays'] = (pd.Timestamp('2026-01-27') - df['FirstTransactionDate']).dt.days


In [20]:
df['Transaction_Type'] = df['Transaction_Type'].str.title().str.strip()


In [21]:
df.head()



,Customer_ID,Satisfaction_Score,Feedback_Comments,Likelihood_to_Recommend,Transaction_ID,Transaction_Date,Transaction_Amount,Transaction_Type,FirstTransactionDate,AccountAgeDays
0,1,10.0,Very satisfied,9,29.0,2023-01-02 04:00:00,156.0,Loan Payment,2023-01-02 04:00:00,1120
1,1,10.0,Very satisfied,9,282.0,2023-01-12 17:00:00,1572.0,Bill Payment,2023-01-02 04:00:00,1120
2,1,10.0,Very satisfied,9,2883.0,2023-05-01 02:00:00,4867.0,Bill Payment,2023-01-02 04:00:00,1120
3,1,10.0,Very satisfied,9,3135.0,2023-05-11 14:00:00,1278.0,Investment,2023-01-02 04:00:00,1120
4,1,10.0,Very satisfied,9,3626.0,2023-06-01 01:00:00,3970.0,Purchase,2023-01-02 04:00:00,1120


In [22]:
df['Transaction_Type'].unique()


array(['Loan Payment', 'Bill Payment', 'Investment', 'Purchase',
       'No Transaction'], dtype=object)

In [23]:
df[['Transaction_Type']].sample(10)


,Transaction_Type
16535,Bill Payment
1631,Investment
18266,Bill Payment
11466,Investment
14200,Bill Payment
12555,Bill Payment
9911,Investment
16777,Investment
7489,Investment
14820,Bill Payment


In [24]:
# Total transaction amount per customer
df_total = df.groupby('Customer_ID')['Transaction_Amount'].sum().reset_index()
df_total.rename(columns={'Transaction_Amount': 'TotalTransactionAmount'}, inplace=True)

# Number of transactions per customer
df_count = df.groupby('Customer_ID')['Transaction_ID'].count().reset_index()
df_count.rename(columns={'Transaction_ID': 'TransactionCount'}, inplace=True)

# Merge back
df = df.merge(df_total, on='Customer_ID', how='left')
df = df.merge(df_count, on='Customer_ID', how='left')


In [31]:
print(df.isnull().sum())


Customer_ID                0
Satisfaction_Score         0
Feedback_Comments          0
Likelihood_to_Recommend    0
Transaction_ID             0
Transaction_Date           0
Transaction_Amount         0
Transaction_Type           0
FirstTransactionDate       0
AccountAgeDays             0
TotalTransactionAmount     0
TransactionCount           0
dtype: int64


In [26]:
# Check how many rows have the placeholder date
print((df['Transaction_Date'] == pd.Timestamp('1970-01-01')).sum())
print((df['FirstTransactionDate'] == pd.Timestamp('1970-01-01')).sum())


34
34


In [27]:
# Check minimum, maximum, and first few rows
print(df['AccountAgeDays'].min())
print(df['AccountAgeDays'].max())
print(df[['Customer_ID', 'FirstTransactionDate', 'AccountAgeDays']].head(10))


915
20480
   Customer_ID FirstTransactionDate  AccountAgeDays
0            1  2023-01-02 04:00:00            1120
1            1  2023-01-02 04:00:00            1120
2            1  2023-01-02 04:00:00            1120
3            1  2023-01-02 04:00:00            1120
4            1  2023-01-02 04:00:00            1120
5            1  2023-01-02 04:00:00            1120
6            2  2023-02-06 04:00:00            1085
7            2  2023-02-06 04:00:00            1085
8            3  2023-02-27 23:00:00            1064
9            4  2023-01-22 15:00:00            1100


In [28]:
# See all unique transaction types
print(df['Transaction_Type'].unique())


['Loan Payment' 'Bill Payment' 'Investment' 'Purchase' 'No Transaction']


In [32]:
# Summary of numeric columns
print(df.describe())

# Summary of categorical columns
print(df['Transaction_Type'].value_counts())


        Customer_ID  Satisfaction_Score  Likelihood_to_Recommend  \
count  25000.000000        25000.000000             25000.000000   
mean     506.820840            5.690313                 5.526560   
min        1.000000            1.000000                 1.000000   
25%      255.000000            3.000000                 3.000000   
50%      509.000000            6.000000                 6.000000   
75%      766.000000            8.000000                 8.000000   
max     1000.000000           60.000000                10.000000   
std      291.870887            3.567779                 2.872188   

       Transaction_ID            Transaction_Date  Transaction_Amount  \
count    25000.000000                       25000        25000.000000   
mean      2507.272200  2023-03-20 02:25:42.384000         3088.543480   
min          0.000000         1970-01-01 00:00:00            0.000000   
25%       1245.000000         2023-02-21 20:00:00         1248.000000   
50%       2516.500000 

In [33]:
df.to_csv("FinMark_Week3_Final.csv", index=False)
